# Importando as bibliotecas necessárias

In [1]:
const faker = require('faker');
const mongoose = require('mongoose');
const _ = require('lodash');
const countryData = require('country-data');
mongoose.set('useCreateIndex', true);

Mongoose {
  connections: [
    NativeConnection {
      base: [Circular],
      collections: {},
      models: {},
      config: [Object],
      replica: false,
      options: null,
      otherDbs: [],
      relatedDbs: {},
      states: [Object: null prototype],
      _readyState: 0,
      _closeCalled: false,
      _hasOpened: false,
      plugins: [],
      id: 0,
      _listening: false
    }
  ],
  models: {},
  modelSchemas: {},
  options: {
    pluralization: true,
    useCreateIndex: true,
    [Symbol(mongoose:default)]: true
  },
  _pluralize: [Function: pluralize],
  Schema: [Function: Schema] {
    reserved: [Object: null prototype] {
      validate: 1,
      toObject: 1,
      schema: 1,
      save: 1,
      remove: 1,
      populated: 1,
      isNew: 1,
      isModified: 1,
      init: 1,
      get: 1,
      errors: 1,
      collection: 1,
      removeListener: 1,
      on: 1,
      listeners: 1,
      emit: 1,
      prototype: 1
    },
    Types: {
      String: [Functio

# Conectando a base de dados usando mongoose

In [2]:
mongoose.connect('mongodb://localhost/effective_store', { useUnifiedTopology: true, useNewUrlParser: true }).then(() => console.log('Connected'));
console.log('Connecting...')

Connecting...
Connected


# Definindo os países disponíveis

In [3]:
const availableCountries = _.filter(countryData.countries.all, countrie => countrie.status === 'assigned');

# Definindo o schema de Invoice

In [4]:
const invoiceSchema = new mongoose.Schema({
    invoiceNumber: String,
    dueDate: Date,
    status: String,
    totalAmount: Number,
    invoiceDoc: [{
        vendorUniqueCode: String,
        docNumber: String,
        productList: Array
    }]
})

# Definindo o schema de Sellers

In [5]:
const sellerSchema = new mongoose.Schema({
  code: {
    type: Number,
    unique: true
  },
  name: String,
  country: {
    code: { type: String, enum: availableCountries.map(country => country.alpha2) },
    name: { type: String, enum: availableCountries.map(country => country.name) },
    emojiFlag: { type: String, enum: availableCountries.map(country => country.emoji) }
  },
  invoice: [{
      invoiceNumber: String
  }],
  createdAt: Date,
  productCatalog: [{
    name: String,
    price: Number
  }]
});

# Definindo o model de Seller

In [6]:
const Seller = mongoose.model('Seller', sellerSchema);

# Gerando dados de sellers

In [7]:
const SELLERS_LENGTH = 500;
const PRODUCTS_LENGTH = 100;

const availableProducts = _.range(PRODUCTS_LENGTH).map(() => ({
  name: faker.commerce.product(),
  price: parseFloat(faker.commerce.price()),
  description: faker.commerce.productDescription(),
}));

const sellers = _.range(SELLERS_LENGTH).map((sellerIndex) => {
  const productCatalogSize = _.random(1, PRODUCTS_LENGTH)
  const country = _.sample(availableCountries)
  const productCatalog = []
  _.range(productCatalogSize).forEach(() => {
    const productSample = _.sample(availableProducts)
    if (!productCatalog.includes(productSample)) {
      productCatalog.push(productSample)
    }
  });
  
  return new Seller({
    code: sellerIndex,
    name: faker.company.companyName(),
    country: { code: country.alpha2, name: country.name, emojiFlag: country.emoji },
    createdAt: faker.date.past(10),
    productCatalog
  });
});

# Populando base de dados

In [8]:
Seller.deleteMany({}, () => console.log('cleared'));
Seller.insertMany(sellers, () => console.log('Inserted'));
console.log('Processing...')

Processing...
cleared
Inserted


# Query 1 - Lojistas (Sellers) com maior número de produtos disponíveis na plataforma

In [9]:
Seller.aggregate([
  {
    $project: {
      _id: 0,
      name: 1,
      description: 1,  
      productCatalogSize: { $cond: { if: { $isArray: "$productCatalog" }, then: { $size: "$productCatalog" }, else: "NA"} }
    }
  },
  {
    $sort : { productCatalogSize : -1 }
  },
  {
    $limit: 10
  }
], (errors, result) => console.table(result))
console.log('Processing...')

Processing...
┌─────────┬─────────────────────────────────┬────────────────────┐
│ (index) │              name               │ productCatalogSize │
├─────────┼─────────────────────────────────┼────────────────────┤
│    0    │  'Walsh, Wilkinson and Reilly'  │         68         │
│    1    │      'Shields - VonRueden'      │         67         │
│    2    │  'Spencer, Hand and Gorczany'   │         66         │
│    3    │          'Mohr Group'           │         66         │
│    4    │       'Champlin - Jones'        │         65         │
│    5    │  'Thiel, Simonis and Reinger'   │         65         │
│    6    │           'Veum Inc'            │         65         │
│    7    │     'Emmerich - Schroeder'      │         64         │
│    8    │  'Dooley, Adams and Franecki'   │         64         │
│    9    │ 'McCullough, Stehr and Hilpert' │         64         │
└─────────┴─────────────────────────────────┴────────────────────┘


# Query 2 - Produtos mais comuns entre os lojistas associados

In [13]:
Seller.aggregate([
  {
    $unwind: '$productCatalog'
  },
  {
    $group: {
      _id: '$productCatalog.name',
      counts: { $sum: 1 }
    }
  },
  {
    $sort: { counts: -1 }
  }
], (errors, result) => console.table(result))
console.log('Processing...')

Processing...
undefined


# Query 3 - Países com mais lojistas

In [23]:
Seller.aggregate([
  {
    $group: {
      _id: '$country.name',
      counts: { $sum: 1 }
    }
  },
  {
    $sort: { counts: -1 }
  },
  {
    $limit: 10
  }
], (errors, result) => console.table(result))
console.log('Processing...')

Processing...
┌─────────┬───────────────────────┬────────┐
│ (index) │          _id          │ counts │
├─────────┼───────────────────────┼────────┤
│    0    │      'Gibraltar'      │   6    │
│    1    │ 'Virgin Islands (US)' │   6    │
│    2    │       'France'        │   6    │
│    3    │  'Saint Barthélemy'   │   5    │
│    4    │     'Guadeloupe'      │   5    │
│    5    │       'Mexico'        │   5    │
│    6    │       'Algeria'       │   5    │
│    7    │   'Cayman Islands'    │   5    │
│    8    │        'Aruba'        │   5    │
│    9    │       'Panama'        │   5    │
└─────────┴───────────────────────┴────────┘


# Query 4 - Produtos com mais lojistas por região

In [22]:
Seller.aggregate([
  {
    $unwind: '$productCatalog'
  },
  {
    $group: {
      _id: '$productCatalog.name',
      country: { $first : "$country.name" },
      counts: { $sum: 1 },
    }
  },
  {
    $sort: { country: -1 }
  }
], (errors, result) => console.table(result))
console.log('Processing...')

Processing...
┌─────────┬────────────┬────────────────────────────┬────────┐
│ (index) │    _id     │          country           │ counts │
├─────────┼────────────┼────────────────────────────┼────────┤
│    0    │ 'Computer' │ 'Virgin Islands (British)' │  545   │
│    1    │   'Fish'   │         'Senegal'          │  339   │
│    2    │ 'Chicken'  │         'Senegal'          │  538   │
│    3    │  'Gloves'  │         'Senegal'          │  402   │
│    4    │  'Towels'  │         'Senegal'          │  368   │
│    5    │  'Bacon'   │         'Senegal'          │  1275  │
│    6    │   'Car'    │         'Senegal'          │  547   │
│    7    │ 'Keyboard' │           'Fiji'           │  561   │
│    8    │   'Soap'   │           'Fiji'           │  745   │
│    9    │  'Salad'   │           'Fiji'           │  1113  │
│   10    │  'Chips'   │           'Fiji'           │  704   │
│   11    │   'Hat'    │           'Fiji'           │  347   │
│   12    │ 'Sausages' │           'Fiji'

# Query 5 - Lista dos lojistas (Sellers) ordenada pelos mais novos

In [21]:
Seller.aggregate([
  {
    $project: {
      _id: 0,
      name: 1,
      createdAt: 1
    }
  },
  {
    $sort: { createdAt: -1 }
  },
  {
    $limit: 10
  }
], (errors, result) => console.table(result))
console.log('Processing...')

Processing...
┌─────────┬──────────────────────────────────┬──────────────────────────┐
│ (index) │               name               │        createdAt         │
├─────────┼──────────────────────────────────┼──────────────────────────┤
│    0    │         'Langosh - Lang'         │ 2020-09-08T08:46:04.426Z │
│    1    │          'Denesik LLC'           │ 2020-09-03T17:03:37.810Z │
│    2    │     'Funk, Raynor and Fadel'     │ 2020-08-23T02:10:04.448Z │
│    3    │    'West, Kling and Fritsch'     │ 2020-08-08T15:36:23.069Z │
│    4    │       'Stamm - Wilderman'        │ 2020-08-01T14:42:46.099Z │
│    5    │        'Kilback - White'         │ 2020-07-18T22:00:30.637Z │
│    6    │ 'Ullrich, Nader and Heidenreich' │ 2020-07-04T09:50:09.776Z │
│    7    │        'Zulauf and Sons'         │ 2020-06-28T04:21:16.961Z │
│    8    │       'Rath - Williamson'        │ 2020-06-27T12:53:37.151Z │
│    9    │      'Schneider - Leffler'       │ 2020-06-26T11:26:51.426Z │
└─────────┴─────────────

# Query 6 - Lista dos lojistas (Sellers) ordenada pelos mais antiga

In [18]:
Seller.aggregate([
  {
    $project: {
      _id: 0,
      name: 1,
      createdAt: 1
    }
  },
  {
    $sort: { createdAt: -1 }
  },
  {
    $limit: 10
  }
], (errors, result) => console.table(result))
console.log('Processing...')

Processing...
┌─────────┬──────────────────────────────────┬──────────────────────────┐
│ (index) │               name               │        createdAt         │
├─────────┼──────────────────────────────────┼──────────────────────────┤
│    0    │         'Langosh - Lang'         │ 2020-09-08T08:46:04.426Z │
│    1    │          'Denesik LLC'           │ 2020-09-03T17:03:37.810Z │
│    2    │     'Funk, Raynor and Fadel'     │ 2020-08-23T02:10:04.448Z │
│    3    │    'West, Kling and Fritsch'     │ 2020-08-08T15:36:23.069Z │
│    4    │       'Stamm - Wilderman'        │ 2020-08-01T14:42:46.099Z │
│    5    │        'Kilback - White'         │ 2020-07-18T22:00:30.637Z │
│    6    │ 'Ullrich, Nader and Heidenreich' │ 2020-07-04T09:50:09.776Z │
│    7    │        'Zulauf and Sons'         │ 2020-06-28T04:21:16.961Z │
│    8    │       'Rath - Williamson'        │ 2020-06-27T12:53:37.151Z │
│    9    │      'Schneider - Leffler'       │ 2020-06-26T11:26:51.426Z │
└─────────┴─────────────